# Amoeba en Borovicka

In [1]:
from scipy import optimize, stats
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm

## Funciones

In [2]:
"""
Funciones de Borovicka que
dependen de [C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0]
"""

def construct_radio(C, A, F, X0, Y0, a0):
    def r(x, y):
        return C * ( np.sqrt( (x-X0)**2 + (y-Y0)**2 ) + A*(y-X0)*np.cos(F-a0) - A*(x-X0)*np.sin(F-a0) )
    return r

def construct_u(C, A, F, V, S, D, P, Q, X0, Y0, a0):
    def u(x, y):
        r = construct_radio(C, A, F, X0, Y0, a0)
        return V*r(x, y) + S*(np.e**(D*r(x,y)) - 1) + P*(np.e**(Q*r(x,y)**2) - 1) 
    return u

def construct_b(E, X0, Y0, a0):
    def b(x, y):
        return a0 - E + np.arctan2((y - Y0),(x - X0))
    return b

def construct_altura(C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0):
    def z(x, y):
        u = construct_u(C, A, F, V, S, D, P, Q, X0, Y0, a0)
        b = construct_b(E, X0, Y0, a0)
        return np.arccos( np.cos(u(x,y))*np.cos(ep) - np.sin(u(x,y))*np.sin(ep)*np.cos(b(x,y)) )
    return z

def construct_altura_deg(C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0):
    def z(x, y):
        u = construct_u(C, A, F, V, S, D, P, Q, X0, Y0, a0)
        b = construct_b(E, X0, Y0, a0)
        return 90 - np.rad2deg( np.arccos( np.cos(u(x,y))*np.cos(ep) - np.sin(u(x,y))*np.sin(ep)*np.cos(b(x,y)) ) )
    return z

def construct_azimuth(C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0):
    def az(x, y):
        u = construct_u(C, A, F, V, S, D, P, Q, X0, Y0, a0)
        b = construct_b(E, X0, Y0, a0)
        z = construct_altura(C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0)
        return np.arcsin( np.sin(b(x,y))*np.sin(u(x,y))/np.sin(z(x,y)) ) + E
    return az

def construct_azimuth_deg(C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0):
    def az(x, y):
        u = construct_u(C, A, F, V, S, D, P, Q, X0, Y0, a0)
        b = construct_b(E, X0, Y0, a0)
        z = construct_altura(C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0)
        return np.rad2deg( np.arcsin( np.sin(b(x,y))*np.sin(u(x,y))/np.sin(z(x,y)) ) + E ) + 90
    return az

"""
Estos son constructores de una funcion de Xi**2
"""

def construct_alt_xi(x, y, z):
    def xi(C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0):
        alt_teorico = construct_altura_deg(C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0)
        return (z - alt_teorico(x, y))**2
    return xi

def construct_az_xi(x, y, z):
    def xi(C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0):
        az_teorico = construct_azimuth_deg(C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0)
        return (z - az_teorico(x, y))**2
    return xi

In [3]:
"""
Para reducir los datos outlayers segun que tan lejos quedan de la primera
aproximacion por Borovicka
"""
def drop_outlayers_by_borovicka(pixtab_df, plano, diff = 1):
    
    X_MIN = 250
    X_MAX = 1250
    Y_MIN = 0
    Y_MAX = 950
    MAX_RADIO = 500
    RADIO_STEPS = 2000
    query_str = "{}<=x<={} and {}<=y<={}".format(X_MIN, X_MAX,
                                                            Y_MIN, Y_MAX)
    data = pixtab_df.dropna(thresh = 7).query(query_str)
    
    pbar = tqdm(total=len(data.x.values), desc="Droping Outlayers")
    new_df_indx = []
    for indx in data.index:
        x = data.at[indx,"x"]
        y = data.at[indx,"y"]
        alt = data.at[indx,"alt"]
        if np.abs(plano(x,y)-alt) < diff:
            new_df_indx.append(indx)
        pbar.update()
    pbar.close()
    return data.loc[new_df_indx]

# Por si queremos exportar los datos para visualizarlos 
# out = drop_outlayers_by_borovicka(data, X0, Y0)
# out
# out["xcentroid"] = out["x"]
# out["ycentroid"] = out["y"]
# out.to_csv("drop_outlayer.dat", sep=' ', index=False)
# plt.gca().invert_yaxis()
# plt.scatter(out.x.values, out.y.values, c=out.alt.values, s=0.01)

In [4]:
"""
Para plotear la altura o el azimut, es necesaria
una lista con todos los parametros 
[C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0]
"""

def plot_plano(plano):
    x = np.linspace(0, 1200, 100)
    y = np.linspace(0, 900, 100)
    X, Y = np.meshgrid(x, y)
    Z = plano(X, Y)
    fig,ax=plt.subplots(1,1)
    cp = ax.contourf(X, Y, Z, 10)
    fig.colorbar(cp) # Add a colorbar to a plot
    ax.set_title('Toda una linea que minimiza y=mx+n')
    plt.gca().invert_yaxis()
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    plt.show()

def plot_altura(vector):
    plano = construct_altura_deg(*vector)
    plot_plano(plano)
    
def plot_azimuth(vector):
    plano = construct_azimuth_deg(*vector)
    plot_plano(plano)

In [5]:
"""
Esto lo definimos por cada intento segun los parametros constantes que
queramos dejar
"""

def construct_acumulate_alt_xi(x_list, y_list, z_list, ctes = None):
    def acumulate_xi(params):
        V, S, D, P, Q, a0 = ctes
        C, A, F, E, ep, X0, Y0 = params
        suma_xi = 0
        for x, y, z in zip(x_list, y_list, z_list):
            xi = construct_alt_xi(int(x), int(y), int(z))
            suma_xi += xi(C, A, F, V, S, D, P, Q, E, ep, float(X0), float(Y0), a0)
        pbar.update()
        return suma_xi
    return acumulate_xi

def construct_acumulate_az_xi(x_list, y_list, z_list, ctes = None):
    def acumulate_xi(params):
        V, S, D, P, Q, a0 = ctes
        C, A, F, E, ep, X0, Y0 = params
        suma_xi = 0
        for x, y, z in zip(x_list, y_list, z_list):
            xi = construct_az_xi(int(x), int(y), int(z))
            suma_xi += xi(C, A, F, V, S, D, P, Q, E, ep, float(X0), float(Y0), a0)
        pbar.update()
        return suma_xii
    return acumulate_xi

######################################################################################################

def amoeba(params, mega_alt_xi, mega_az_xi):
    new_alt_params = optimize.fmin(mega_alt_xi, params, maxfun=1100)
    new_az_params = optimize.fmin(mega_az_xi, params, maxfun=1100)
    return [new_alt_params, new_az_params]

## Reduction types:

There are many types as mentioned in the section 2.2.3. of the paper.
However, IRED may be zero when entering REDSKY. Then the standard
reduction type 314 is used. The most usual types are

314 - V,S,D,P,Q kept at their standard values, other constants computed

334 - P,Q kept, C=1, other computed

354 - C=1, other computed (requires stars close to horizon)

311 - as 314 but A=0, F=0 (similarly 331, 351)

310 - as 311 but also epsilon=0, E=0 (similarly 330, 350) ([V,S,D,P,Q]=ctes, [A,F,E,ep]=0, solo se busca C)

319 - as 314 but epsilon,E,A,F kept at their initial values (similarly
      339, 359) ([V,S,D,P,Q,A,F,E,ep]=ctes solo se busca C)

## Algunos parametros conocidos
C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0 :


- <sub> $Xi^2=2.206$ : [0.04884978986903654, 0.020304825896093035, 0.35248628545828664, 0.0682171007, 0.00310608468, -5.75141386e-05, 4.26865327e-06, -0.00415889519, 0.005383738310580881, 0.0014950622892165601, 724, 472, -3*np.pi/4]

- <sub>  $Xi^2=2.154$ : [4.37875766e-02,  5.19853314e-03,  1.05416567e+00,  7.33610329e-02, -5.24112025e-02, -1.39156740e-01,  4.41635759e-03,  4.71369350e-03, 2.74820813e+00, -1.02665758e-01, 731, 506, -0.582*np.pi] 

Encontrados ahora:
- [1)](#section1) <sub> $Xi^2=2.210$ : [0.048826087003725455, -0.021142174069559417, 0.32598730818211297, 0.0682171007, 0.00310608468, -5.75141386e-05, 4.26865327e-06, -0.00415889519, 0.08741648813678929, -0.0022711518934522366, 724, 472, np.pi/4] 

- [2)](#section2) <sub> $Xi^2=2.246$ : [0.048826087003725455, -0.021142174069559417, 0.32598730818211297, 0.0682171007, 0.00310608468, -5.75141386e-05, 4.26865327e-06, -0.00415889519, 0.08741648813678929, -0.0022711518934522366, 724.59618727, 472.68750979, np.pi/4]

- [3)](#section3) <sub> $Xi^2_{alt}=2.228$, $Xi^2_{az}=60.30$ : [0.048826087003725455, -0.021142174069559417, 0.32598730818211297, 0.0682171007, 0.00310608468, -5.75141386e-05, 4.26865327e-06, -0.00415889519, 0.08741648813678929, -0.0022711518934522366, 724.59618727, 472.68750979, 0.82635545]

- [4)](#section4) <sub> $Xi^2_{alt}=2.205$, $Xi^2_{az}=20.94$ : [0.048826087003725455, -0.021142174069559417, 0.32598730818211297, 0.0682171007, 0.00310608468, -5.75141386e-05, 4.26865327e-06, -0.00415889519, -0.0298275, -0.0022711518934522366, 724, 472, 0.75060687]
    
- [5)](#section5) <sub> $Xi^2_{alt}=3.120$, $Xi^2_{az}=15.75$ : [0.048826087003725455, -0.021142174069559417, 0.32598730818211297, 0.0682171007, 0.00310608468, -5.75141386e-05, 4.26865327e-06, -0.00415889519, -4.82552710e-03, -0.0022711518934522366, 7.28125200e+02,  4.78181845e+02, 7.73853421e-01]
  
- [6)](#section6) <sub> $Xi^2_{alt}=2.320$, $Xi^2_{az}=15.75$ : [0.04952671,  0.00310034, -1.04782185, 0.0682171007, 0.00310608468, -5.75141386e-05, 4.26865327e-06, -0.00415889519, -4.82552710e-03, -0.0022711518934522366, 7.28125200e+02,  4.78181845e+02, 7.73853421e-01]

- [7)](#section7) <sub> $Xi^2_{alt}=2.236$, $Xi^2_{az}=15.75$ : [0.04952671,  0.00310034, -1.04782185, 9.61696411e-02, -3.11311558e+00,  8.47643783e-03,  5.83185714e-04, 7.66455100e-03, -4.82552710e-03, -0.0022711518934522366, 7.28125200e+02,  4.78181845e+02, 7.73853421e-01]
    
- [8)](#section8) <sub> $Xi^2_{alt}=2.142$, $Xi^2_{az}=15.89$ : [0.04933449,  0.00478405, -0.1298804, 9.61696411e-02, -3.11311558e+00,  8.47643783e-03,  5.83185714e-04, 7.66455100e-03, -4.82552710e-03, -0.0022711518934522366, 7.28125200e+02,  4.78181845e+02, 7.73853421e-01]
    
- [9)](#section9) <sub> $Xi^2_{alt}=2.147$, $Xi^2_{az}=15.89$ : [0.04933449,  0.00478405, -0.1298804, 9.44302225e-02, -3.90770908e+00,  6.47675936e-03,  1.99616127e-03, 5.31560995e-03, -4.82552710e-03, -0.0022711518934522366, 7.28125200e+02,  4.78181845e+02, 7.73853421e-01]


Facts:
- mover ep reduciendo azimut es super peligroso
- hay que sumarle 90 al azimuth porque borovicka lo tiene entre \[-90,90\]
- E no solo rota, si no, que tambien ensancha o agudiza el espacio que ocupan los grados
- mover E,X0,Y0,a0 juntos reduciendo azimuth es muy bueno para encontrar el centro
- mover V,S,D,P,Q no afecta al azimuth!



[Greedy](#greedy) Intentamos reducir altura y azimut al mismo tiempo,
los paramtros iniciales fueron los obtenidos en 6

[subir](#pipeline)<a id='pipeline'></a>
## Intentemos un pipeline

Solo podriamos conocer un V,S,D,P,Q aproximado dado del paper de Borovicka. Ademas sobre C,A,F podemos seguir el consejo de hacer A,F nulos, pero C tendra que ser un valor pequeño como 0.01. Para E,ep podemos hacerlos nulos. El centro X0,Y0 tendremos que aproximarlo al ojo, tener una idea, en este caso 724,472. Por ultimo el offset del azimut a0 podemos hacerlo nulo.

El principal problema es reducir los datos en un principio porque no sabemos qué es lo que estamos descartando
El paso 2-3 es bueno separarlo para hacer la reduccion mas eficientemente

Parametros iniciales:
- [0)] <sub> $Xi^2_{alt}=46.65$, $Xi^2_{az}=1629$ : [1, 0, 0, 0.003, 0.0001, 0.00001, 0, 0, 0, 0.001, 724, 472, 0]

Luego el orden que mejor nos resultó fué:
- [0)](#paso0) Reducir altura con C,A,F usando TODOS los datos (00:31:41) <sub> $Xi^2_{alt}=2.19$, $Xi^2_{az}=2632$ : [1.11380716e+00, -2.00584578e-02, -5.65478506e-01,  3.00000000e-03, 1.00000000e-04, 1.00000000e-05,  0.00000000e+00,  0.00000000e+00, 0.00000000e+00,  1.00000000e-03,  7.24000000e+02,  4.72000000e+02, 0.00000000e+00]

- [1)](#paso1) Reducir altura con C,A,F,E,ep usando datos discriminados (0:47:46) <sub> $Xi^2_{alt}=2.22$, $Xi^2_{az}=4045$ : [1.11754514e+00, -1.07052342e-02,  1.61991333e-01,  3.00000000e-03, 1.00000000e-04,  1.00000000e-05,  0.00000000e+00,  0.00000000e+00, 5.30758288e-01, -1.59562151e-02,  7.24000000e+02,  4.72000000e+02, 0.00000000e+00]
    
- [2)](#paso2) Reducir azimut con a0 usando datos discriminados (0:08:08) <sub> $Xi^2_{alt}=3.47$, $Xi^2_{az}=923.8$ : [1.11754514e+00, -1.07052342e-02,  1.61991333e-01,  3.00000000e-03, 1.00000000e-04,  1.00000000e-05,  0.00000000e+00,  0.00000000e+00, 5.30758288e-01, -1.59562151e-02,  7.24000000e+02,  4.72000000e+02, 1.12192725e+00]
    
- [3)](#paso3) Reducir azimut con E,X0,Y0,a0 usando datos discriminados (0:34:56) <sub> $Xi^2_{alt}=2.59$, $Xi^2_{az}=15.74$ : [1.11754514e+00, -1.07052342e-02,  1.61991333e-01,  3.00000000e-03, 1.00000000e-04,  1.00000000e-05,  0.00000000e+00,  0.00000000e+00, -5.12369350e-03, -1.59562151e-02,  7.25674086e+02,  4.80521222e+02, 7.73607375e-01]
    
- [4)](#paso4) Reducir altura con V,S,D,P,Q usando datos discriminados (1:00:13) <sub> $Xi^2_{alt}=2.58$, $Xi^2_{az}=15.74$ : [1.11754514e+00, -1.07052342e-02,  1.61991333e-01,  2.99975896e-03, 9.85888749e-05,  1.03115518e-05,  4.51458709e-08,  4.62043644e-05, -5.12369350e-03, -1.59562151e-02,  7.25674086e+02,  4.80521222e+02, 7.73607375e-01]
    
- [5)](#paso5) Reducir altura con C,A,F usando datos discriminados (0:25:18) <sub> $Xi^2_{alt}=2.32$, $Xi^2_{az}=15.72$ : [1.12405645e+00,  4.50987250e-03, -1.34218133e+00,  2.99975896e-03, 9.85888749e-05,  1.03115518e-05,  4.51458709e-08,  4.62043644e-05, -5.12369350e-03, -1.59562151e-02,  7.25674086e+02,  4.80521222e+02, 7.73607375e-01]
    
Demoró como 3 horas y media y el resultado fue regular

In [19]:
results_list = []

[subir](#pipeline)<a id='paso0'></a>
## Paso 0

In [37]:
"""
314 - V, S, D, P, Q, X0, Y0, a0
"""

def construct_acumulate_alt_xi(x_list, y_list, z_list, ctes = None):
    def acumulate_xi(params):
        V, S, D, P, Q, E, ep, X0, Y0, a0 = ctes
        C, A, F, = params
        suma_xi = 0
        for x, y, z in zip(x_list, y_list, z_list):
            xi = construct_alt_xi(int(x), int(y), int(z))
            suma_xi += xi(C, A, F, V, S, D, P, Q, E, ep, float(X0), float(Y0), a0)
        pbar.update()
        return suma_xi
    return acumulate_xi

def construct_acumulate_az_xi(x_list, y_list, z_list, ctes = None):
    def acumulate_xi(params):
        V, S, D, P, Q, E, ep, X0, Y0, a0 = ctes
        C, A, F, = params
        suma_xi = 0
        for x, y, z in zip(x_list, y_list, z_list):
            xi = construct_az_xi(int(x), int(y), int(z))
            suma_xi += xi(C, A, F, V, S, D, P, Q, E, ep, float(X0), float(Y0), a0)
        pbar.update()
        return suma_xi
    return acumulate_xi

######################################################################################################

def amoeba(params, mega_alt_xi, mega_az_xi):
    new_alt_params = optimize.fmin(mega_alt_xi, params, maxfun=2200)
#     new_az_params = optimize.fmin(mega_az_xi, params, maxfun=1100)
    return [new_alt_params]

init_params = [1, 0, 0]
ctes = [0.003, 0.0001, 0.00001, 0, 0,
           0, 0.001, 724, 472, 0]

# init_params = [*result_2[:3]]
# ctes = [*result_2[3:]]

data = pd.read_csv("Data.pixtab", sep=" ")
data = data.dropna()
# data = drop_outlayers_by_borovicka(data, construct_altura_deg(*init_params, *ctes), 5)
x_exp = data.x.values
y_exp = data.y.values
alt_catalogo = data.alt.values
az_catalogo = np.rad2deg(np.arccos(np.cos(np.deg2rad(data.az.values))))
mega_alt_xi = construct_acumulate_alt_xi(x_exp, y_exp, alt_catalogo, ctes)
mega_az_xi = construct_acumulate_az_xi(x_exp, y_exp, az_catalogo, ctes)

pbar = tqdm(total=2200, desc="Aproximando altura y azimuth")
results_list += amoeba(init_params, mega_alt_xi, mega_az_xi)
pbar.close()
results_list[-1]

result_0 = np.array([*results_list[-1], *ctes])
result_0


Aproximando altura y azimuth:   6%|██▉                                            | 135/2200 [12:26<3:02:54,  5.31s/it]


Aproximando altura y azimuth:  12%|█████▊                                         | 271/2200 [24:26<2:45:59,  5.16s/it]


Aproximando altura y azimuth:  16%|███████▌                                       | 355/2200 [31:41<2:40:09,  5.21s/it]

Optimization terminated successfully.
         Current function value: 178080.255622
         Iterations: 199
         Function evaluations: 355


Aproximando altura y azimuth:  16%|███████▌                                       | 355/2200 [31:41<2:44:40,  5.36s/it]


array([ 1.11380716e+00, -2.00584578e-02, -5.65478506e-01,  3.00000000e-03,
        1.00000000e-04,  1.00000000e-05,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  1.00000000e-03,  7.24000000e+02,  4.72000000e+02,
        0.00000000e+00])

In [ ]:
temp_result_0 = np.array([ 1.11380716e+00, -2.00584578e-02, -5.65478506e-01,  3.00000000e-03,
        1.00000000e-04,  1.00000000e-05,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  1.00000000e-03,  7.24000000e+02,  4.72000000e+02,
        0.00000000e+00])

[subir](#pipeline)<a id='paso1'></a>
### Paso 1
Primero haremos un acercamiento 314 para buscar parametros C,A,F,E,ep que nos ayuden a reducir mejor los datos outlayers

Otra opcion era buscar E,X0,Y0,a0 desde el principio sin quitar datos outlayers, el problema es que E,ep al ser 0 tienen problemas en la funcion de azimut

In [40]:
"""
314 - V, S, D, P, Q, X0, Y0, a0
"""

def construct_acumulate_alt_xi(x_list, y_list, z_list, ctes = None):
    def acumulate_xi(params):
        V, S, D, P, Q, X0, Y0, a0 = ctes
        C, A, F, E, ep, = params
        suma_xi = 0
        for x, y, z in zip(x_list, y_list, z_list):
            xi = construct_alt_xi(int(x), int(y), int(z))
            suma_xi += xi(C, A, F, V, S, D, P, Q, E, ep, float(X0), float(Y0), a0)
        pbar.update()
        return suma_xi
    return acumulate_xi

def construct_acumulate_az_xi(x_list, y_list, z_list, ctes = None):
    def acumulate_xi(params):
        V, S, D, P, Q, X0, Y0, a0 = ctes
        C, A, F, E, ep, = params
        suma_xi = 0
        for x, y, z in zip(x_list, y_list, z_list):
            xi = construct_az_xi(int(x), int(y), int(z))
            suma_xi += xi(C, A, F, V, S, D, P, Q, E, ep, float(X0), float(Y0), a0)
        pbar.update()
        return suma_xi
    return acumulate_xi

######################################################################################################

def amoeba(params, mega_alt_xi, mega_az_xi):
    new_alt_params = optimize.fmin(mega_alt_xi, params, maxfun=1100)
#     new_az_params = optimize.fmin(mega_az_xi, params, maxfun=1100)
    return [new_alt_params]

init_params = [*result_0[:3], *result_0[8:10]]
ctes = [*result_0[3:8],*result_0[10:]]

# init_params = [0.01, 0, 0, 0, 0]
# ctes = [0.03, 0.007, 0.09, 2.2*10**-6, 0.006,
#            724, 472, 0]

data = pd.read_csv("Data.pixtab", sep=" ")
data = data.dropna()
data = drop_outlayers_by_borovicka(data, construct_altura_deg(*init_params[:3], *ctes[:-3], *init_params[-2:], *ctes[-3:]), 5)
x_exp = data.x.values
y_exp = data.y.values
alt_catalogo = data.alt.values
az_catalogo = np.rad2deg(np.arccos(np.cos(np.deg2rad(data.az.values))))
mega_alt_xi = construct_acumulate_alt_xi(x_exp, y_exp, alt_catalogo, ctes)
mega_az_xi = construct_acumulate_az_xi(x_exp, y_exp, az_catalogo, ctes)

pbar = tqdm(total=1100, desc="Aproximando altura y azimuth")
results_list += amoeba(init_params, mega_alt_xi, mega_az_xi)
pbar.close()
results_list[-1]

result_1 = np.array([*results_list[-1][:3], *ctes[:-3], *results_list[-1][-2:], *ctes[-3:]])
result_1


Droping Outlayers: 100%|███████████████████████████████████████████████████████| 80771/80771 [00:12<00:00, 6531.21it/s]

Aproximando altura y azimuth:   1%|▌                                               | 14/1100 [01:12<1:34:42,  5.23s/it]


Aproximando altura y azimuth:  14%|██████▍                                        | 150/1100 [12:28<1:20:00,  5.05s/it]


Aproximando altura y azimuth:  26%|████████████▏                                  | 286/1100 [23:39<1:07:04,  4.94s/it]


Aproximando altura y azimuth:  38%|██████████████████▊                              | 422/1100 [34:41<54:55,  4.86s/it]


Aproximando altura y azimuth:  51%|████████████████████████▊                        | 558/1100 [45:45<44:04,  4.88s/it]


Aproximando altura y azimuth:  53%|█████████████████████████▉                       | 583/1100 [47:46<41:15,  4.79s/it]

Optimization terminated successfully.
         Current function value: 38147.044725
         Iterations: 364
         Function evaluations: 583


Aproximando altura y azimuth:  53%|█████████████████████████▉                       | 583/1100 [47:46<42:21,  4.92s/it]


array([ 1.11754514e+00, -1.07052342e-02,  1.61991333e-01,  3.00000000e-03,
        1.00000000e-04,  1.00000000e-05,  0.00000000e+00,  0.00000000e+00,
        5.30758288e-01, -1.59562151e-02,  7.24000000e+02,  4.72000000e+02,
        0.00000000e+00])

In [ ]:
temp_result_1 = np.array([ 1.11754514e+00, -1.07052342e-02,  1.61991333e-01,  3.00000000e-03,
        1.00000000e-04,  1.00000000e-05,  0.00000000e+00,  0.00000000e+00,
        5.30758288e-01, -1.59562151e-02,  7.24000000e+02,  4.72000000e+02,
        0.00000000e+00])

[subir](#pipeline)<a id='paso2'></a>
### Paso 2
Ahora buscamos a0 quitando datos outlayers reduciendo azimut. Al ser solo a0 deberia ser mas fácil.

In [43]:
"""
311 - C, A, F, V, S, D, P, Q, ep, a0
"""

def construct_acumulate_alt_xi(x_list, y_list, z_list, ctes = None):
    def acumulate_xi(params):
        C, A, F, V, S, D, P, Q, E, ep, X0, Y0 = ctes
        a0, = params
        suma_xi = 0
        for x, y, z in zip(x_list, y_list, z_list):
            xi = construct_alt_xi(int(x), int(y), int(z))
            suma_xi += xi(C, A, F, V, S, D, P, Q, E, ep, float(X0), float(Y0), a0)
        pbar.update()
        return suma_xi
    return acumulate_xi

def construct_acumulate_az_xi(x_list, y_list, z_list, ctes = None):
    def acumulate_xi(params):
        C, A, F, V, S, D, P, Q, E, ep, X0, Y0 = ctes
        a0, = params
        suma_xi = 0
        for x, y, z in zip(x_list, y_list, z_list):
            xi = construct_az_xi(int(x), int(y), int(z))
            suma_xi += xi(C, A, F, V, S, D, P, Q, E, ep, float(X0), float(Y0), a0)
        pbar.update()
        return suma_xi
    return acumulate_xi

######################################################################################################

def amoeba(params, mega_alt_xi, mega_az_xi):
#     new_alt_params = optimize.fmin(mega_alt_xi, params, maxfun=1100)
    new_az_params = optimize.fmin(mega_az_xi, params, maxfun=1100)
    return [new_az_params]

init_params = [result_1[-1]]
ctes = [*result_1[:-1]]

data = pd.read_csv("Data.pixtab", sep=" ")
data = data.dropna()
data = drop_outlayers_by_borovicka(data, construct_altura_deg(*ctes, *init_params), 5)
x_exp = data.x.values
y_exp = data.y.values
alt_catalogo = data.alt.values
az_catalogo = np.rad2deg(np.arccos(np.cos(np.deg2rad(data.az.values))))
mega_alt_xi = construct_acumulate_alt_xi(x_exp, y_exp, alt_catalogo, ctes)
mega_az_xi = construct_acumulate_az_xi(x_exp, y_exp, az_catalogo, ctes)

pbar = tqdm(total=1100, desc="Aproximando altura y azimuth")
results_list += amoeba(init_params, mega_alt_xi, mega_az_xi)
pbar.close()
results_list[-1]

result_2 = np.array([*ctes, *results_list[-1]])
result_2


Droping Outlayers: 100%|███████████████████████████████████████████████████████| 80771/80771 [00:12<00:00, 6544.93it/s]

Aproximando altura y azimuth:   1%|▌                                               | 13/1100 [01:39<2:19:37,  7.71s/it]


Aproximando altura y azimuth:   6%|██▊                                             | 64/1100 [08:08<2:10:28,  7.56s/it]

Optimization terminated successfully.
         Current function value: 74035382.161158
         Iterations: 32
         Function evaluations: 64


Aproximando altura y azimuth:   6%|██▊                                             | 64/1100 [08:08<2:11:51,  7.64s/it]


array([ 1.11754514e+00, -1.07052342e-02,  1.61991333e-01,  3.00000000e-03,
        1.00000000e-04,  1.00000000e-05,  0.00000000e+00,  0.00000000e+00,
        5.30758288e-01, -1.59562151e-02,  7.24000000e+02,  4.72000000e+02,
        1.12192725e+00])

In [ ]:
temp_result_2 = np.array([ 1.11754514e+00, -1.07052342e-02,  1.61991333e-01,  3.00000000e-03,
        1.00000000e-04,  1.00000000e-05,  0.00000000e+00,  0.00000000e+00,
        5.30758288e-01, -1.59562151e-02,  7.24000000e+02,  4.72000000e+02,
        1.12192725e+00])

[subir](#pipeline)<a id='paso3'></a>
### Paso 3
Ahora buscamos E,X0,Y0,a0 quitando datos outlayers reduciendo azimut. Como a0 esta más cerca del resultado pude que se demore menos. (19min menos y mejor resultado)

In [46]:
"""
311 - C, A, F, V, S, D, P, Q, ep, a0
"""

def construct_acumulate_alt_xi(x_list, y_list, z_list, ctes = None):
    def acumulate_xi(params):
        C, A, F, V, S, D, P, Q, ep = ctes
        E, X0, Y0, a0, = params
        suma_xi = 0
        for x, y, z in zip(x_list, y_list, z_list):
            xi = construct_alt_xi(int(x), int(y), int(z))
            suma_xi += xi(C, A, F, V, S, D, P, Q, E, ep, float(X0), float(Y0), a0)
        pbar.update()
        return suma_xi
    return acumulate_xi

def construct_acumulate_az_xi(x_list, y_list, z_list, ctes = None):
    def acumulate_xi(params):
        C, A, F, V, S, D, P, Q, ep = ctes
        E, X0, Y0, a0, = params
        suma_xi = 0
        for x, y, z in zip(x_list, y_list, z_list):
            xi = construct_az_xi(int(x), int(y), int(z))
            suma_xi += xi(C, A, F, V, S, D, P, Q, E, ep, float(X0), float(Y0), a0)
        pbar.update()
        return suma_xi
    return acumulate_xi

######################################################################################################

def amoeba(params, mega_alt_xi, mega_az_xi):
#     new_alt_params = optimize.fmin(mega_alt_xi, params, maxfun=1100)
    new_az_params = optimize.fmin(mega_az_xi, params, maxfun=1100)
    return [new_az_params]

init_params = [result_2[8], *result_2[-3:]]
ctes = [*result_2[:8], result_2[9]]

data = pd.read_csv("Data.pixtab", sep=" ")
data = data.dropna()
data = drop_outlayers_by_borovicka(data, construct_altura_deg(*ctes[:-1], init_params[0], ctes[-1], *init_params[-3:]), 5)
x_exp = data.x.values
y_exp = data.y.values
alt_catalogo = data.alt.values
az_catalogo = np.rad2deg(np.arccos(np.cos(np.deg2rad(data.az.values))))
mega_alt_xi = construct_acumulate_alt_xi(x_exp, y_exp, alt_catalogo, ctes)
mega_az_xi = construct_acumulate_az_xi(x_exp, y_exp, az_catalogo, ctes)

pbar = tqdm(total=1100, desc="Aproximando altura y azimuth")
results_list += amoeba(init_params, mega_alt_xi, mega_az_xi)
pbar.close()
results_list[-1]

result_3 = np.array([*ctes[:-1], results_list[-1][0], ctes[-1], *results_list[-1][-3:]])
result_3


Droping Outlayers: 100%|███████████████████████████████████████████████████████| 80771/80771 [00:12<00:00, 6475.85it/s]

Aproximando altura y azimuth:   1%|▌                                               | 13/1100 [01:35<2:11:20,  7.25s/it]


Aproximando altura y azimuth:  14%|██████▎                                        | 149/1100 [18:40<1:58:33,  7.48s/it]


Aproximando altura y azimuth:  25%|███████████▉                                   | 278/1100 [34:56<1:43:23,  7.55s/it]

Optimization terminated successfully.
         Current function value: 1215985.908735
         Iterations: 159
         Function evaluations: 278


Aproximando altura y azimuth:  25%|███████████▉                                   | 278/1100 [34:56<1:43:17,  7.54s/it]


array([ 1.11754514e+00, -1.07052342e-02,  1.61991333e-01,  3.00000000e-03,
        1.00000000e-04,  1.00000000e-05,  0.00000000e+00,  0.00000000e+00,
       -5.12369350e-03, -1.59562151e-02,  7.25674086e+02,  4.80521222e+02,
        7.73607375e-01])

In [ ]:
temp_result_3 = np.array([ 1.11754514e+00, -1.07052342e-02,  1.61991333e-01,  3.00000000e-03,
        1.00000000e-04,  1.00000000e-05,  0.00000000e+00,  0.00000000e+00,
       -5.12369350e-03, -1.59562151e-02,  7.25674086e+02,  4.80521222e+02,
        7.73607375e-01])

[subir](#pipeline)<a id='paso4'></a>
### Paso 4
Ahora buscamos V,S,D,P,Q reduciendo la altura. Según lo que experimentamos no deberia afectar al azimut

Algo particular es que si reducimos discriminando datos apunta mal, debe ser que necesita mas datos en el horizonte

In [49]:
"""
314 - V, S, D, P, Q
"""

def construct_acumulate_alt_xi(x_list, y_list, z_list, ctes = None):
    def acumulate_xi(params):
        C, A, F, E, ep, X0, Y0, a0 = ctes
        V, S, D, P, Q, = params
        suma_xi = 0
        for x, y, z in zip(x_list, y_list, z_list):
            xi = construct_alt_xi(int(x), int(y), int(z))
            suma_xi += xi(C, A, F, V, S, D, P, Q, E, ep, float(X0), float(Y0), a0)
        pbar.update()
        return suma_xi
    return acumulate_xi

def construct_acumulate_az_xi(x_list, y_list, z_list, ctes = None):
    def acumulate_xi(params):
        C, A, F, E, ep, X0, Y0, a0 = ctes
        V, S, D, P, Q, = params
        suma_xi = 0
        for x, y, z in zip(x_list, y_list, z_list):
            xi = construct_az_xi(int(x), int(y), int(z))
            suma_xi += xi(C, A, F, V, S, D, P, Q, E, ep, float(X0), float(Y0), a0)
        pbar.update()
        return suma_xi
    return acumulate_xi

######################################################################################################

def amoeba(params, mega_alt_xi, mega_az_xi):
    new_alt_params = optimize.fmin(mega_alt_xi, params, maxfun=1100)
#     new_az_params = optimize.fmin(mega_az_xi, params, maxfun=1100)
    return [new_alt_params]


init_params = [*result_3[3:8]]
ctes = [*result_3[:3], *result_3[8:]]

data = pd.read_csv("Data.pixtab", sep=" ")
data = data.dropna()
data = drop_outlayers_by_borovicka(data, construct_altura_deg(*ctes[:3],*init_params, *ctes[3:]), 5)
x_exp = data.x.values
y_exp = data.y.values
alt_catalogo = data.alt.values
az_catalogo = np.rad2deg(np.arccos(np.cos(np.deg2rad(data.az.values))))
mega_alt_xi = construct_acumulate_alt_xi(x_exp, y_exp, alt_catalogo, ctes)
mega_az_xi = construct_acumulate_az_xi(x_exp, y_exp, az_catalogo, ctes)

pbar = tqdm(total=1100, desc="Aproximando altura y azimuth")
results_list += amoeba(init_params, mega_alt_xi, mega_az_xi)
pbar.close()
results_list[-1]

result_4 = np.array([*ctes[:3],*results_list[-1], *ctes[3:]])
result_4


Droping Outlayers: 100%|███████████████████████████████████████████████████████| 80771/80771 [00:12<00:00, 6514.40it/s]

Aproximando altura y azimuth:   1%|▌                                               | 13/1100 [01:05<1:31:39,  5.06s/it]


Aproximando altura y azimuth:  14%|██████▎                                        | 149/1100 [12:28<1:18:56,  4.98s/it]


Aproximando altura y azimuth:  26%|████████████▏                                  | 285/1100 [23:42<1:07:55,  5.00s/it]


Aproximando altura y azimuth:  38%|██████████████████▊                              | 421/1100 [35:06<56:09,  4.96s/it]


Aproximando altura y azimuth:  51%|████████████████████████▊                        | 557/1100 [46:36<45:42,  5.05s/it]


Aproximando altura y azimuth:  63%|██████████████████████████████▊                  | 693/1100 [57:58<33:55,  5.00s/it]


Aproximando altura y azimuth:  65%|██████████████████████████████▊                | 720/1100 [1:00:13<31:41,  5.00s/it]

Optimization terminated successfully.
         Current function value: 56346.107123
         Iterations: 432
         Function evaluations: 720


Aproximando altura y azimuth:  65%|██████████████████████████████▊                | 720/1100 [1:00:13<31:47,  5.02s/it]


array([ 1.11754514e+00, -1.07052342e-02,  1.61991333e-01,  2.99975896e-03,
        9.85888749e-05,  1.03115518e-05,  4.51458709e-08,  4.62043644e-05,
       -5.12369350e-03, -1.59562151e-02,  7.25674086e+02,  4.80521222e+02,
        7.73607375e-01])

In [ ]:
temp_result_4 = np.array([ 1.11754514e+00, -1.07052342e-02,  1.61991333e-01,  2.99975896e-03,
        9.85888749e-05,  1.03115518e-05,  4.51458709e-08,  4.62043644e-05,
       -5.12369350e-03, -1.59562151e-02,  7.25674086e+02,  4.80521222e+02,
        7.73607375e-01])

[subir](#pipeline)<a id='paso5'></a>
## Paso 5
Ahora buscamos C,A,F reduciendo la altura. Según lo que experimentamos no deberia afectar al azimut

In [52]:
"""
314 - V, S, D, P, Q, X0, Y0, a0
"""

def construct_acumulate_alt_xi(x_list, y_list, z_list, ctes = None):
    def acumulate_xi(params):
        V, S, D, P, Q, E, ep, X0, Y0, a0 = ctes
        C, A, F, = params
        suma_xi = 0
        for x, y, z in zip(x_list, y_list, z_list):
            xi = construct_alt_xi(int(x), int(y), int(z))
            suma_xi += xi(C, A, F, V, S, D, P, Q, E, ep, float(X0), float(Y0), a0)
        pbar.update()
        return suma_xi
    return acumulate_xi

def construct_acumulate_az_xi(x_list, y_list, z_list, ctes = None):
    def acumulate_xi(params):
        V, S, D, P, Q, E, ep, X0, Y0, a0 = ctes
        C, A, F, = params
        suma_xi = 0
        for x, y, z in zip(x_list, y_list, z_list):
            xi = construct_az_xi(int(x), int(y), int(z))
            suma_xi += xi(C, A, F, V, S, D, P, Q, E, ep, float(X0), float(Y0), a0)
        pbar.update()
        return suma_xi
    return acumulate_xi

######################################################################################################

def amoeba(params, mega_alt_xi, mega_az_xi):
    new_alt_params = optimize.fmin(mega_alt_xi, params, maxfun=1100)
#     new_az_params = optimize.fmin(mega_az_xi, params, maxfun=1100)
    return [new_alt_params]

init_params = [*result_4[:3]]
ctes = [*result_4[3:]]

data = pd.read_csv("Data.pixtab", sep=" ")
data = data.dropna()
data = drop_outlayers_by_borovicka(data, construct_altura_deg(*init_params, *ctes), 5)
x_exp = data.x.values
y_exp = data.y.values
alt_catalogo = data.alt.values
az_catalogo = np.rad2deg(np.arccos(np.cos(np.deg2rad(data.az.values))))
mega_alt_xi = construct_acumulate_alt_xi(x_exp, y_exp, alt_catalogo, ctes)
mega_az_xi = construct_acumulate_az_xi(x_exp, y_exp, az_catalogo, ctes)

pbar = tqdm(total=1100, desc="Aproximando altura y azimuth")
results_list += amoeba(init_params, mega_alt_xi, mega_az_xi)
pbar.close()
results_list[-1]

result_5 = np.array([*results_list[-1], *ctes])
result_5



Droping Outlayers:   0%|                                                                     | 0/80771 [00:00<?, ?it/s]

Droping Outlayers:   1%|▍                                                        | 640/80771 [00:00<00:12, 6353.68it/s]

Droping Outlayers:   1%|▊                                                       | 1190/80771 [00:00<00:13, 6055.81it/s]

Droping Outlayers:   2%|█▎                                                      | 1836/80771 [00:00<00:12, 6159.20it/s]

Droping Outlayers:   3%|█▋                                                      | 2497/80771 [00:00<00:12, 6274.79it/s]

Droping Outlayers:   4%|██▏                                                     | 3092/80771 [00:00<00:12, 6159.66it/s]

Droping Outlayers:   5%|██▌                                                     | 3691/80771 [00:00<00:12, 6094.15it/s]

Droping Outlayers:   5%|██▉                                                     | 4275/80771 [00:00<00:12, 6001.75it/s]

Droping Outlayers:   6%|███▍  

Aproximando altura y azimuth:   1%|▍                                               | 10/1100 [00:50<1:30:56,  5.01s/it]

Aproximando altura y azimuth:   1%|▍                                               | 11/1100 [00:55<1:30:36,  4.99s/it]

Aproximando altura y azimuth:   1%|▌                                               | 12/1100 [00:59<1:30:24,  4.99s/it]

Aproximando altura y azimuth:   1%|▌                                               | 13/1100 [01:04<1:30:11,  4.98s/it]

Aproximando altura y azimuth:   1%|▌                                               | 14/1100 [01:09<1:29:57,  4.97s/it]

Aproximando altura y azimuth:   1%|▋                                               | 15/1100 [01:14<1:29:46,  4.96s/it]

Aproximando altura y azimuth:   1%|▋                                               | 16/1100 [01:19<1:30:31,  5.01s/it]

Aproximando altura y azimuth:   2%|▋                                               | 17/1100 [01:24<1:30:26,  5.01s/it]

Aproximando altura y azimuth:   

Aproximando altura y azimuth:  13%|██████▏                                        | 144/1100 [12:01<1:19:05,  4.96s/it]

Aproximando altura y azimuth:  13%|██████▏                                        | 145/1100 [12:06<1:18:47,  4.95s/it]

Aproximando altura y azimuth:  13%|██████▏                                        | 146/1100 [12:11<1:18:30,  4.94s/it]

Aproximando altura y azimuth:  13%|██████▎                                        | 147/1100 [12:16<1:19:17,  4.99s/it]

Aproximando altura y azimuth:  13%|██████▎                                        | 148/1100 [12:21<1:18:50,  4.97s/it]

Aproximando altura y azimuth:  14%|██████▎                                        | 149/1100 [12:26<1:18:28,  4.95s/it]

Aproximando altura y azimuth:  14%|██████▍                                        | 150/1100 [12:31<1:18:07,  4.93s/it]

Aproximando altura y azimuth:  14%|██████▍                                        | 151/1100 [12:35<1:17:53,  4.92s/it]

Aproximando altura y azimuth:  1

Aproximando altura y azimuth:  25%|███████████▉                                   | 278/1100 [22:53<1:05:49,  4.80s/it]

Aproximando altura y azimuth:  25%|███████████▉                                   | 279/1100 [22:58<1:05:40,  4.80s/it]

Aproximando altura y azimuth:  25%|███████████▉                                   | 280/1100 [23:02<1:05:37,  4.80s/it]

Aproximando altura y azimuth:  26%|████████████                                   | 281/1100 [23:07<1:05:34,  4.80s/it]

Aproximando altura y azimuth:  26%|████████████                                   | 282/1100 [23:12<1:05:28,  4.80s/it]

Aproximando altura y azimuth:  26%|████████████                                   | 283/1100 [23:17<1:05:39,  4.82s/it]

Aproximando altura y azimuth:  26%|████████████▏                                  | 284/1100 [23:22<1:05:40,  4.83s/it]

Aproximando altura y azimuth:  26%|████████████▏                                  | 285/1100 [23:27<1:05:44,  4.84s/it]

Aproximando altura y azimuth:  2

Optimization terminated successfully.
         Current function value: 39985.433687
         Iterations: 170
         Function evaluations: 308


Aproximando altura y azimuth:  28%|█████████████▏                                 | 308/1100 [25:18<1:05:04,  4.93s/it]


array([ 1.12405645e+00,  4.50987250e-03, -1.34218133e+00,  2.99975896e-03,
        9.85888749e-05,  1.03115518e-05,  4.51458709e-08,  4.62043644e-05,
       -5.12369350e-03, -1.59562151e-02,  7.25674086e+02,  4.80521222e+02,
        7.73607375e-01])

In [ ]:
temp_result_5 = np.array([ 1.12405645e+00,  4.50987250e-03, -1.34218133e+00,  2.99975896e-03,
        9.85888749e-05,  1.03115518e-05,  4.51458709e-08,  4.62043644e-05,
       -5.12369350e-03, -1.59562151e-02,  7.25674086e+02,  4.80521222e+02,
        7.73607375e-01])

# Desarrollo

In [ ]:
"""
314 - V, S, D, P, Q, X0, Y0, a0
"""

def construct_acumulate_alt_xi(x_list, y_list, z_list, ctes = None):
    def acumulate_xi(params):
        V, S, D, P, Q, X0, Y0, a0 = ctes
        C, A, F, E, ep, = params
        suma_xi = 0
        for x, y, z in zip(x_list, y_list, z_list):
            xi = construct_alt_xi(int(x), int(y), int(z))
            suma_xi += xi(C, A, F, V, S, D, P, Q, E, ep, float(X0), float(Y0), a0)
        pbar.update()
        return suma_xi
    return acumulate_xi

def construct_acumulate_az_xi(x_list, y_list, z_list, ctes = None):
    def acumulate_xi(params):
        V, S, D, P, Q, X0, Y0, a0 = ctes
        C, A, F, E, ep, = params
        suma_xi = 0
        for x, y, z in zip(x_list, y_list, z_list):
            xi = construct_az_xi(int(x), int(y), int(z))
            suma_xi += xi(C, A, F, V, S, D, P, Q, E, ep, float(X0), float(Y0), a0)
        pbar.update()
        return suma_xi
    return acumulate_xi

######################################################################################################

def amoeba(params, mega_alt_xi, mega_az_xi):
    new_alt_params = optimize.fmin(mega_alt_xi, params, maxfun=1100)
#     new_az_params = optimize.fmin(mega_az_xi, params, maxfun=1100)
    return [new_alt_params]

In [ ]:
init_params = [0.01, 0, 0, 0, 0]
ctes = [0.03, 0.007, 0.09, 2.2*10**-6, 0.006,
           724, 472, 0]

data = pd.read_csv("Data.pixtab", sep=" ")
data = data.dropna()
data = drop_outlayers_by_borovicka(data, construct_altura_deg(*init_params[:3], *ctes[:-3], *init_params[-2:], *ctes[-3:]), 5)
x_exp = data.x.values
y_exp = data.y.values
alt_catalogo = data.alt.values
az_catalogo = np.rad2deg(np.arccos(np.cos(np.deg2rad(data.az.values))))
mega_alt_xi = construct_acumulate_alt_xi(x_exp, y_exp, alt_catalogo, ctes)
mega_az_xi = construct_acumulate_az_xi(x_exp, y_exp, az_catalogo, ctes)
results_list = []

In [ ]:
pbar = tqdm(total=2200, desc="Aproximando altura y azimuth")
results_list += amoeba(init_params, mega_alt_xi, mega_az_xi)
pbar.close()
results_list[-1]

In [1]:
# result_3 = (0.09935315, -0.03387904, -0.04467695,
#              0.03, 0.007, 0.09, 2.2*10**-6, 0.006,
#             0.01311418, -0.00192129, 724, 472, 0)
plot_altura(result_3)
plot_azimuth(result_3)

SyntaxError: invalid syntax (<ipython-input-1-db77e6054b64>, line 4)

In [ ]:
data = pd.read_csv("Data.pixtab", sep=" ")
data = data.dropna()
# data = drop_outlayers_by_borovicka(data, construct_altura_deg(*init_params[:3], *ctes[:-3], *init_params[-2:], *ctes[-3:]), 5)
x_exp = data.x.values
y_exp = data.y.values
alt_catalogo = data.alt.values
az_catalogo = np.rad2deg(np.arccos(np.cos(np.deg2rad(data.az.values))))
mega_alt_xi = construct_acumulate_alt_xi(x_exp, y_exp, alt_catalogo, ctes)
mega_az_xi = construct_acumulate_az_xi(x_exp, y_exp, az_catalogo, ctes)

old_params = np.array([0.01, 0, 0, 0, 0])
new_params = np.array([0.09935315, -0.03387904, -0.04467695,  0.01311418, -0.00192129])
(mega_alt_xi(old_params)/len(x_exp), mega_alt_xi(new_params)/len(x_exp))

In [ ]:
plt.gca().invert_yaxis()
plt.scatter(x_exp, y_exp, c=alt_catalogo, s=0.01)

In [ ]:
data

In [ ]:
"""
311 - C, A, F, V, S, D, P, Q, ep, a0
"""

def construct_acumulate_alt_xi(x_list, y_list, z_list, ctes = None):
    def acumulate_xi(params):
        C, A, F, V, S, D, P, Q, ep = ctes
        E, X0, Y0, a0, = params
        suma_xi = 0
        for x, y, z in zip(x_list, y_list, z_list):
            xi = construct_alt_xi(int(x), int(y), int(z))
            suma_xi += xi(C, A, F, V, S, D, P, Q, E, ep, float(X0), float(Y0), a0)
        pbar.update()
        return suma_xi
    return acumulate_xi

def construct_acumulate_az_xi(x_list, y_list, z_list, ctes = None):
    def acumulate_xi(params):
        C, A, F, V, S, D, P, Q, ep = ctes
        E, X0, Y0, a0, = params
        suma_xi = 0
        for x, y, z in zip(x_list, y_list, z_list):
            xi = construct_az_xi(int(x), int(y), int(z))
            suma_xi += xi(C, A, F, V, S, D, P, Q, E, ep, float(X0), float(Y0), a0)
        pbar.update()
        return suma_xi
    return acumulate_xi

######################################################################################################

def amoeba(params, mega_alt_xi, mega_az_xi):
#     new_alt_params = optimize.fmin(mega_alt_xi, params, maxfun=1100)
    new_az_params = optimize.fmin(mega_az_xi, params, maxfun=1100)
    return [new_az_params]

In [ ]:
init_params = [0, 724, 472, 0]
ctes = [0.01, 0, 0,
         0.03, 0.007, 0.09, 2.2*10**-6, 0.006,
         0]

data = pd.read_csv("Data.pixtab", sep=" ")
data = data.dropna()
# data = drop_outlayers_by_borovicka(data, construct_altura_deg(*ctes[:-1], init_params[0], ctes[-1], *init_params[-3:]), 5)
x_exp = data.x.values
y_exp = data.y.values
alt_catalogo = data.alt.values
az_catalogo = np.rad2deg(np.arccos(np.cos(np.deg2rad(data.az.values))))
mega_alt_xi = construct_acumulate_alt_xi(x_exp, y_exp, alt_catalogo, ctes)
mega_az_xi = construct_acumulate_az_xi(x_exp, y_exp, az_catalogo, ctes)
results_list = []

In [ ]:
pbar = tqdm(total=1100, desc="Aproximando altura y azimuth")
results_list += amoeba(init_params, mega_alt_xi, mega_az_xi)
pbar.close()
results_list[-1]

In [ ]:
result_5 = (0.048826087003725455, -0.021142174069559417, 0.32598730818211297,
             0.0682171007, 0.00310608468, -5.75141386e-05, 4.26865327e-06,
             -0.00415889519, -4.82552710e-03, -0.0022711518934522366, 7.28125200e+02,  4.78181845e+02, 7.73853421e-01)
plot_altura(result_5)
plot_azimuth(result_5)

In [53]:
"""
314 - V, S, D, P, Q, X0, Y0, a0
"""

def construct_acumulate_alt_xi(x_list, y_list, z_list, ctes = None):
    def acumulate_xi(params):
        C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0 = params
        suma_xi = 0
        for x, y, z in zip(x_list, y_list, z_list):
            xi = construct_alt_xi(int(x), int(y), int(z))
            suma_xi += xi(C, A, F, V, S, D, P, Q, E, ep, float(X0), float(Y0), a0)
        pbar.update()
        return suma_xi
    return acumulate_xi

def construct_acumulate_az_xi(x_list, y_list, z_list, ctes = None):
    def acumulate_xi(params):
        C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0 = params
        suma_xi = 0
        for x, y, z in zip(x_list, y_list, z_list):
            xi = construct_az_xi(int(x), int(y), int(z))
            suma_xi += xi(C, A, F, V, S, D, P, Q, E, ep, float(X0), float(Y0), a0)
        pbar.update()
        return suma_xi
    return acumulate_xi

data = pd.read_csv("Data.pixtab", sep=" ")
data = data.dropna()
# data = drop_outlayers_by_borovicka(data, construct_altura_deg(*ctes[:3],*init_params, *ctes[3:]), 5)
x_exp = data.x.values
y_exp = data.y.values
alt_catalogo = data.alt.values
az_catalogo = np.rad2deg(np.arccos(np.cos(np.deg2rad(data.az.values))))
mega_alt_xi = construct_acumulate_alt_xi(x_exp, y_exp, alt_catalogo)
mega_az_xi = construct_acumulate_az_xi(x_exp, y_exp, az_catalogo)

print("alt: ",mega_alt_xi(result_4)/len(x_exp), mega_alt_xi(result_5)/len(x_exp))
print("az: ",mega_az_xi(result_4)/len(x_exp), mega_az_xi(result_5)/len(x_exp))

alt:  2.588337328337103 2.3245927042139014
az:  15.748233516555729 15.72720900373798
